In [3]:
import os
import sys
sys.path.append(os.path.abspath("..")) # set working dir

from web3 import Web3
from src.config import ALCHEMY_API_KEY

ETHEREUM_RPC_URL = f"https://eth-mainnet.g.alchemy.com/v2/{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(ETHEREUM_RPC_URL))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block) # latest block

True
AttributeDict({'hash': HexBytes('0x1a6cda84664ba44c1b0387f4b040032c8e7ff6ccc4664762ba7655cf7f3320c4'), 'parentHash': HexBytes('0xa7d16558040bf445f1f83cae083ce91447f1fab31cf96cf98850ec833be9757c'), 'sha3Uncles': HexBytes('0x1dcc4de8dec75d7aab85b567b6ccd41ad312451b948a7413f0a142fd40d49347'), 'miner': '0x4838B106FCe9647Bdf1E7877BF73cE8B0BAD5f97', 'stateRoot': HexBytes('0x6b17082d5bca86c84447226843c9dbe7ef04f7b49b1e41bb94d59b836eac14d9'), 'transactionsRoot': HexBytes('0xb8a3973b694eb5e46320646de42e3d815297d92b6cb7004dd834dd4e783851b6'), 'receiptsRoot': HexBytes('0x37c1b64b184bc78b79c71795c73549fd6c9fc38f45d3e5f1e48c650eb72f8244'), 'logsBloom': HexBytes('0x8bf9edf7c1edbf9f757bbdf6fff5defd7ff465ff0ddbf9bdb2dfb9ffffbef796ffdfc776bbfebd9ef7dbff7bfbfb5ffe77f7cf5ecf7efdf5df9dbeda7baffffffef6feffbadfffefed5beffffdfd7ffeffdfdefff77f7eff7cf5cdd7dfee9ff6ffffebdfaefbdffdeeb8ddd7eff77fdbbfafff7eeffffefffaffc77d7bfafff7fbfa6fdfffbf7faafcddfd7fef6fb87ff473ff7dfbbfbfffbdbdf6d6efbef7bebffb7bbbd7f17f7

# Setup

In [4]:
from src.utils.retrieveAbi import save_abi_to_file
from src.config import MAINNET_UNISWAP_V2_ROUTER_02, MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET

# Load ABIs
save_abi_to_file(MAINNET_UNISWAP_V2_ROUTER_02) # uniswap v2 router abi
save_abi_to_file(MAINNET_SUSHISWAP_ROUTER_ADDRESS) # sushiswap router abi

ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/1_0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D_abi.json
ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/1_0xd9e1cE17f2641f24aE83637ab66a2cca9C378B9F_abi.json


# Contract Interactions

In [5]:
import time
from src.utils.retrieveAbi import load_abi

def get_amounts_out(token1_address, token2_address, amount_in, router_contract):
    """Uses w3 getAmountsOut function to return amounts out, given two tokens, router contract and amount in"""
    token1_address = Web3.to_checksum_address(token1_address)
    token2_address = Web3.to_checksum_address(token2_address)
    path = [token1_address, token2_address]
    amounts_out = router_contract.functions.getAmountsOut(amount_in, path).call()
    return w3.from_wei(amounts_out[-1], 'ether')

In [6]:
# uniswap
uniswap_router_abi = load_abi(MAINNET_UNISWAP_V2_ROUTER_02, CHAINID_MAINNET)
uniswap_router_address = Web3.to_checksum_address(MAINNET_UNISWAP_V2_ROUTER_02)
uniswap_router_contract = w3.eth.contract(address=uniswap_router_address, abi=uniswap_router_abi)
uniswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
uniswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
uniswap_amount_in = w3.to_wei(1, "ether")

# sushiswap
sushiswap_router_abi = load_abi(MAINNET_SUSHISWAP_ROUTER_ADDRESS, CHAINID_MAINNET)
sushiswap_router_address = Web3.to_checksum_address(MAINNET_SUSHISWAP_ROUTER_ADDRESS)
sushiswap_router_contract = w3.eth.contract(address=sushiswap_router_address, abi=sushiswap_router_abi)
sushiswap_weth_address = "0xC02aaa39b223FE8D0A0e5C4F27eAD9083C756Cc2"
sushiswap_dai_address = "0x6B175474E89094C44Da98b954EedeAC495271d0F"
sushiswap_amount_in = w3.to_wei(1, "ether")

try:
    prev_block=0
    while True:
        block = w3.eth.block_number
        uniswap_amount_out = get_amounts_out(uniswap_weth_address, uniswap_dai_address, uniswap_amount_in, uniswap_router_contract)
        sushiswap_amount_out = get_amounts_out(sushiswap_weth_address, sushiswap_dai_address, sushiswap_amount_in, sushiswap_router_contract)
        if block > prev_block:
            print(f"Block {block} | Uniswap: 1 ETH ≈ {uniswap_amount_out} DAI | Sushiswap: 1 ETH ≈ {sushiswap_amount_out} | Delta = {uniswap_amount_out-sushiswap_amount_out}")
            prev_block = block
        time.sleep(1)
except KeyboardInterrupt:
    print("Stopped by user.")

Block 23012168 | Uniswap: 1 ETH ≈ 3821.839405525931914452 DAI | Sushiswap: 1 ETH ≈ 3821.366829372867942771 | Delta = 0.472576153063971681
Stopped by user.


# Binance API

In [8]:
from binance.client import Client
from src.config import BINANCE_API_KEY, BINANCE_API_SECRET

In [9]:
client = Client(BINANCE_API_KEY, BINANCE_API_SECRET)

def get_order_book(symbol='ETHUSDC', limit=100):
    """
    Holt das Orderbuch für ein Handelspaar (symbol).
    limit bestimmt, wie viele Bids und Asks abgerufen werden.
    """
    order_book = client.get_order_book(symbol=symbol, limit=limit)
    bids = order_book['bids']  # Liste von [Preis, Menge]
    asks = order_book['asks']

    print(f"Orderbuch für {symbol}:")
    print("Top Bids:")
    for price, qty in bids:
        print(f"Preis: {price}, Menge: {qty}")
    print("Top Asks:")
    for price, qty in asks:
        print(f"Preis: {price}, Menge: {qty}")

In [10]:
get_order_book()

Orderbuch für ETHUSDC:
Top Bids:
Preis: 3847.10000000, Menge: 0.00130000
Preis: 3847.00000000, Menge: 1.34000000
Preis: 3846.96000000, Menge: 0.00130000
Preis: 3846.93000000, Menge: 0.00680000
Preis: 3846.92000000, Menge: 3.91050000
Preis: 3846.90000000, Menge: 1.34000000
Preis: 3846.84000000, Menge: 0.00260000
Preis: 3846.79000000, Menge: 0.00280000
Preis: 3846.74000000, Menge: 0.00480000
Preis: 3846.73000000, Menge: 2.46430000
Preis: 3846.72000000, Menge: 0.00130000
Preis: 3846.71000000, Menge: 0.59800000
Preis: 3846.61000000, Menge: 1.71370000
Preis: 3846.60000000, Menge: 0.00260000
Preis: 3846.55000000, Menge: 0.00400000
Preis: 3846.53000000, Menge: 3.27500000
Preis: 3846.52000000, Menge: 0.40000000
Preis: 3846.49000000, Menge: 2.60700000
Preis: 3846.48000000, Menge: 4.10120000
Preis: 3846.47000000, Menge: 4.96680000
Preis: 3846.46000000, Menge: 15.77430000
Preis: 3846.44000000, Menge: 0.40000000
Preis: 3846.43000000, Menge: 0.19000000
Preis: 3846.42000000, Menge: 0.80000000
Preis:

# Unichain

In [11]:
from src.utils.retrieveAbi import save_abi_to_file, load_abi
from src.config import (
    UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, CHAINID_UNICHAIN,
    UNICHAIN_UNISWAP_V2_ROUTER_02, UNICHAIN_WETH, UNICHAIN_USDC,
    ALCHEMY_UNICHAIN_BASE_RPC_URL)

save_abi_to_file(UNICHAIN_UNIVERSAL_ROUTER_V2_ADDRESS, chain_id=CHAINID_UNICHAIN)
save_abi_to_file(UNICHAIN_UNISWAP_V2_ROUTER_02, chain_id=CHAINID_UNICHAIN)

ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/130_0xEf740bf23aCaE26f6492B10de645D6B98dC8Eaf3_abi.json
ABI saved to: /home/tobias/personal-dex-trading/notebooks/../out/130_0x284F11109359a7e1306C3e447ef14D38400063FF_abi.json


In [14]:
WETH_AMOUNT_IN = 0.01
USDC_AMOUNT_IN = 38
weth_wei_amount_in = w3.to_wei(WETH_AMOUNT_IN, "ether")
usdc_amounts_in = USDC_AMOUNT_IN * 10 ** 6 # USDC has 6 decimals

router_address = Web3.to_checksum_address(UNICHAIN_UNISWAP_V2_ROUTER_02)
router_abi = load_abi(router_address, CHAINID_UNICHAIN)
router_contract = w3.eth.contract(address=router_address, abi=router_abi)

alchemy_rpc_url = f"{ALCHEMY_UNICHAIN_BASE_RPC_URL}{ALCHEMY_API_KEY}"
w3 = Web3(Web3.HTTPProvider(alchemy_rpc_url))
print(w3.is_connected()) # check connection
latest_block = w3.eth.get_block("latest")
print(latest_block.number) # latest block number

True
22891319


In [ ]:
amounts_out = router_contract.functions.getAmountsOut(weth_wei_amount_in, [UNICHAIN_WETH, UNICHAIN_USDC]).call()
print(f"Input Amount (WETH): {WETH_AMOUNT_IN} WETH")
print(f"Output Amount (USDC): {amounts_out[-1] / (10 ** 6)} USDC")  # USDC has 6 decimals
print(f"----")
amounts_out_reverse = router_contract.functions.getAmountsOut(usdc_amounts_in, [UNICHAIN_USDC, UNICHAIN_WETH]).call()
print(f"Input Amount (USDC): {USDC_AMOUNT_IN} USDC")
print(f"Output Amount (WETH): {w3.from_wei(amounts_out_reverse[-1], "ether")} WETH ")  # USDC has 6 decimals

Input Amount (WETH): 0.01 WETH
Output Amount (USDC): 38.065715 USDC
----
Input Amount (USDC): 38 USDC
Output Amount (WETH): 0.009818539196402442 WETH 


Next, we can either do:
- arbitrage between Uniswap v2 and Binance 


- arbitrage between Uniswap v2 and v4 pools